# Neural network test

In [1]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

In [2]:
import pandas as pd
from pandas import DataFrame
from time import sleep
REFRACTORY_PERIOD = 0.1   # 0.1 seconds

In [3]:
# 每個 ESP8266模組 各代表一個 neurons
# neurons = ['neuron_x1', 'neuron_x2', 'neuron_h1', 'neuron_h2', 'neuron_h3', 'neuron_y'] 
neurons = ['neuron_x1', 'neuron_x2'] 

## Start client

In [4]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

My name is Client_366

[connected: ('192.168.0.100', 9662)]
Sending 305 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-01-31 12:38:00.133700'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-01-31 12:38:00.133700'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

[Listen to messages]

Data received: 401 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:00.133700'), ('message_id', '2017-01-31 12:38:00.221700'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_f1d30800': "('192.168.0.103', 10567)", 'Client_366': "('192.168.0.100', 53530)", 'NodeMCU_1dsc000': "('192.168.0.101', 11347)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-31 12:38:00.221700'), ('type', 'result')])

Node not ready yet.


## Utilities

### List nodes

In [5]:
# Ask Hub for a list of connected nodes
def list_nodes():
    message = {'type': 'command',
                       'command': 'list connections by name',
                       'need_result': True}     

    _, asynch_result = the_client.request('Hub', message) 

    try:
        remote_nodes = sorted(list(asynch_result.get().keys()))

        print ('\n[____________ Connected nodes ____________]\n')        
        print('\nConnected nodes:\n{}\n'.format(remote_nodes)) 
        
        return remote_nodes

    except Exception as e:
        print(e)

In [6]:
def reset_node(node):
    message = {'type': 'exec',
               'to_exec': 'import machine;machine.reset()'}
    the_client.request(node, message) 

In [7]:
# reset_node('neuron_x1')

In [8]:
def rename_node(node, new_name):
    message = {'type': 'function',
               'function': 'rename',
               'kwargs': {'name': new_name}}
    the_client.request(node, message) 
    
    message = {'type': 'function',
               'function': 'set_connection_name'}
    the_client.request(node, message) 
    

def rename_nodes(nodes, neurons):    
    i = 0 
    for node in nodes:
        if node != the_client.node.worker.name:  # exclude client self
            rename_node(node, neurons[i])
            i += 1

In [9]:
def fire(node):
    message = {'type': 'function',
               'function': 'fire'}
    the_client.request(node, message) 

def addConnection(node, neuron):
    message = {'type': 'function',
               'function': 'addConnection',
               'kwargs': {'neuron_id': neuron}}
    the_client.request(node, message) 
    
def set_connections(node, connections):
    message = {'type': 'function',
               'function': 'setConnections',
               'kwargs': {'connections': connections}}
    the_client.request(node, message)     
    
def get_connections(node):
    message = {'type': 'function',
               'function': 'getConnections', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()    

def setWeight(node, neuron, weight):
    message = {'type': 'function',
               'function': 'setWeight',
               'kwargs': {'neuron_id': neuron,
                          'weight': weight,}}
    the_client.request(node, message) 

def setThreshold(node, threshold):
    message = {'type': 'function',
               'function': 'setThreshold',
               'kwargs': {'threshold': threshold}}
    the_client.request(node, message) 
        
def getConfig(node):
    message = {'type': 'function',
               'function': 'getConfig', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()

def getLog(node):
    message = {'type': 'function',
               'function': 'getLog', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()

def emptyLog(node):
    message = {'type': 'function',
               'function': 'emptyLog'}
    the_client.request(node, message)
    
def emptyLogs():
    for neuron in neurons:
        emptyLog(neuron) 
        
# 彙整logs。將所有 neurons 中的 logs merge 在一起，成為一個 Pandas.DataFrame
def mergeLogs():
    logs = []
    
    for neuron in neurons:
        if neuron != the_client.node.worker.name:  # exclude client self
            currentLog = getLog(neuron)
            if currentLog:
                logs += currentLog 
            
    df = DataFrame(list(logs), columns = ['time', 'neuron', 'message']) 
    df.set_index('time', inplace = True)
    df.sort_index(inplace = True)
    
    return df        

In [10]:
# print 出 一個 neuron 中的 Log
def printConfig(neuron):
    print('{0:_^78}\n {1}\n'.format(neuron + " config:", getConfig(neuron)))

## 設定 network config

### Rename nodes

In [11]:
remote_nodes = list_nodes() 
rename_nodes(remote_nodes, neurons)
time.sleep(2)
remote_nodes = list_nodes()

Sending 276 bytes
Message:
OrderedDict([('command', 'list connections by name'), ('correlation_id', '2017-01-31 12:38:01.958900'), ('message_id', '2017-01-31 12:38:01.958900'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])


Data received: 401 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:01.958900'), ('message_id', '2017-01-31 12:38:02.114900'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_f1d30800': "('192.168.0.103', 10567)", 'Client_366': "('192.168.0.100', 53530)", 'NodeMCU_1dsc000': "('192.168.0.101', 11347)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-31 12:38:02.114900'), ('type', 'result')])


[____________ Connected nodes ____________]


Connected nodes:
['Client_366', 'NodeMCU_1dsc000', 'NodeMCU_f1d30800']

Sending 284 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:02.196900'), ('function', 'rename'), ('kwargs', {'name': 'neuron_x1'}), ('m

### 清空 log files

In [12]:
# 清除所有 neurons 中的 logs  
emptyLogs()

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:04.913700'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:04.913700'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:05.005700'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:05.005700'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])



### 設定 connections

In [13]:
addConnection('neuron_x1', 'neuron_x2')
getConfig('neuron_x1')

Sending 290 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:05.232700'), ('function', 'addConnection'), ('kwargs', {'neuron_id': 'neuron_x2'}), ('message_id', '2017-01-31 12:38:05.232700'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 269 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:05.293300'), ('function', 'getConfig'), ('message_id', '2017-01-31 12:38:05.293300'), ('need_result', True), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])


Data received: 384 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:05.293300'), ('message_id', '157739'), ('receiver', 'Client_366'), ('reply_to', 'neuron_x1'), ('result', {'connections': {'neuron_x2': 'neuron_x2'}, 'inputs': {}, 'output': {'lasting': 99.99997, 'polarized_time': 135041, 'value': 0}}), ('sender', 'neuron_x1'), ('time_stamp', '2017-01-31 12:38:05.536300'), ('typ

{'connections': {'neuron_x2': 'neuron_x2'},
 'inputs': {},
 'output': {'lasting': 99.99997, 'polarized_time': 135041, 'value': 0}}

### 設定 weights

In [14]:
# hidden layer
setWeight('neuron_x2', 'neuron_x1', 1)  # 設定 neuron_x -> neuron_h1 的 weight = 1

Sending 299 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:05.737900'), ('function', 'setWeight'), ('kwargs', {'neuron_id': 'neuron_x1', 'weight': 1}), ('message_id', '2017-01-31 12:38:05.737900'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])



### 設定 thresholds

In [15]:
# input layer 
setThreshold('neuron_x2', 0.9)

Sending 281 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:05.977100'), ('function', 'setThreshold'), ('kwargs', {'threshold': 0.9}), ('message_id', '2017-01-31 12:38:05.977100'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])



### 模擬 sensor input，然後查看各 neurons 的 output 狀態
一個 neuron fire 之後，如果沒有持續的輸入可維持 fire 的狀態，則過 5 秒鐘之 neuron 的 output 一定為 0

In [16]:
fire('neuron_x1')

Sending 243 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:06.159500'), ('function', 'fire'), ('message_id', '2017-01-31 12:38:06.159500'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])



In [17]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零 
mergeLogs()  # 彙整 logs

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:06.387900'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:06.387900'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:06.481900'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:06.481900'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 266 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:06.739300'), ('function', 'getLog'), ('message_id', '2017-01-31 12:38:06.739300'), ('need_result', True), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])


Data received: 244 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:06.739300'), ('message_id', '159197'), ('receiver', 'Client_366'), ('reply_to', 'neuron_x1'),

,neuron,message
time,,
85728,neuron_x2,neuron_x1 is kicking neuron_x2.
85933,neuron_x2,neuron_x2 fires.
85936,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.


In [18]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零
fire('neuron_x1') # force neuron x1 output 1 and fire.
mergeLogs()  # 彙整 logs

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:09.109700'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:09.109700'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:09.194300'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:09.194300'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 243 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:09.381500'), ('function', 'fire'), ('message_id', '2017-01-31 12:38:09.381500'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 266 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:09.443900'), ('function', 'getLog'), ('message_id', '2017-01-31 12:38:09.443900'), ('need_result', True), ('receiver', 'neuron

,neuron,message
time,,
88868,neuron_x2,neuron_x1 is kicking neuron_x2.
89073,neuron_x2,neuron_x2 fires.
89075,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.
162027,neuron_x1,neuron_x1 fires.
162029,neuron_x1,Setting output of neuron_x1 to ACTION_POTENTIAL.


In [19]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零
fire('neuron_x2') # force neuron x2 output 1 and fire.
mergeLogs()  # 彙整 logs

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:11.596300'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:11.596300'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:11.694300'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:11.694300'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 243 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:11.923100'), ('function', 'fire'), ('message_id', '2017-01-31 12:38:11.923100'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 266 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:11.969900'), ('function', 'getLog'), ('message_id', '2017-01-31 12:38:11.969900'), ('need_result', True), ('receiver', 'neuron

,neuron,message
time,,
91213,neuron_x2,neuron_x2 fires.
91216,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.


In [20]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零
fire('neuron_x1') # force neuron x1 output 1 and fire.
fire('neuron_x2') # force neuron x2 output 1 and fire.
mergeLogs()  # 彙整 logs

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:13.775300'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:13.775300'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 247 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:13.892500'), ('function', 'emptyLog'), ('message_id', '2017-01-31 12:38:13.892500'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 243 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:14.126500'), ('function', 'fire'), ('message_id', '2017-01-31 12:38:14.126500'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])

Sending 243 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:14.188900'), ('function', 'fire'), ('message_id', '2017-01-31 12:38:14.188900'), ('receiver', 'neuron_x2'), ('reply_to', 'Clie

,neuron,message
time,,
93516,neuron_x2,neuron_x2 fires.
93519,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.
93635,neuron_x2,neuron_x1 is kicking neuron_x2.
93840,neuron_x2,neuron_x2 fires.
93842,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.
166836,neuron_x1,neuron_x1 fires.
166838,neuron_x1,Setting output of neuron_x1 to ACTION_POTENTIAL.


In [21]:
for neuron in reversed(neurons): printConfig(neuron)

Sending 269 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:16.117100'), ('function', 'getConfig'), ('message_id', '2017-01-31 12:38:16.117100'), ('need_result', True), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'function')])


Data received: 455 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 12:38:16.117100'), ('message_id', '95223'), ('receiver', 'Client_366'), ('reply_to', 'neuron_x2'), ('result', {'threshold': 0.8999998, 'inputs': {'neuron_x1': {'lasting': 500.0, 'kick_time': 93635, 'value': 1}}, 'weights': {'neuron_x1': 1}, 'output': {'lasting': 99.99997, 'polarized_time': 93842, 'value': 1}}), ('sender', 'neuron_x2'), ('time_stamp', '2017-01-31 12:38:16.399700'), ('type', 'result')])

______________________________neuron_x2 config:_______________________________
 {'threshold': 0.8999998, 'inputs': {'neuron_x1': {'lasting': 500.0, 'kick_time': 93635, 'value': 1}}, 'weights': {'neuron_x1': 1}, 'output': {'l

### Stop the demo

In [22]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')


[________________ Demo stopped ________________]

